# SAM 2023 : TP Requêtes multi-sources pour l'open data

Séances 9 et 10

Objectif : Comprendre et réaliser des jointures entre deux sources réelles distantes lorsque l'accès aux sources est conforme aux standards de l'_open data_.

Contexte : On considère deux sources réelles ainsi que des exemples de requêtes pouvant être posées sur chacune d'entre elles.

On vous demande de mettre en oeuvre des scénarios de requêtes **MULTI** sources.


## Préparation

**Copier** ce notebook dans votre drive et l'exécuter dans l'environnement colab de google


In [1]:
%pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


### Manipuler le résultat d'une requête

Fonctions **tableauResultat** pour extraire le résultat d'une requête


In [2]:
import pandas as pd

# pd.set_option('max_columns', None)
pd.set_option("max_colwidth", None)


# Déterminer la liste des noms d'attributs du résultat, en tenant compte des attributs optionnels.
def schemaResultat(resultats, limit=100):
    d = {}
    for elt in resultats[:limit]:
        for pos, nom in enumerate(elt):
            sum_pos, nb = d.get(nom, (0, 0))
            d[nom] = (sum_pos + pos, nb + 1)
    mean_pos_by_name = [(sum_pos / nb, nom) for (nom, (sum_pos, nb)) in d.items()]
    colonnes = [nom for (mean_pos, nom) in sorted(mean_pos_by_name)]
    return colonnes


def tableauResultat(resultats, limit=100, colonnes=None):
    if colonnes is None:
        colonnes = schemaResultat(resultats, limit)
    tableau = []
    for elt in resultats[:limit]:
        row = {name: v.value for name, v in elt.items()}
        tableau.append([row.get(n, "") for n in colonnes])
    return tableau


print("fonctions définies")


fonctions définies


Afficher le résultat


In [3]:
def afficherResultat(resultats, limit=100):
    print(len(resultats), "éléments")
    colonnes = schemaResultat(resultats, limit)
    print("schéma: ", colonnes)
    result = tableauResultat(resultats, limit, colonnes)
    return pd.DataFrame(result, columns=colonnes)


print("fonctions définies")


fonctions définies


### Importer le pilote pour accéder aux sources


In [4]:
from SPARQLWrapper import SPARQLWrapper2

print("pilote importé")


pilote importé


### Préparer l'accès à la source BNF


In [5]:
# Connexion à la BNF
BNF = SPARQLWrapper2("http://data.bnf.fr/sparql")

BNF_prefix = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rda: <http://rdaregistry.info/Elements/w/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX bnf: <http://data.bnf.fr/ontology/bnf-onto/>

PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX voc2: <http://rdvocab.info/ElementsGr2/>
"""
# print(BNF_prefix)


### Préparer l'accès à la source DBPEDIA


In [6]:
# Connexion à DBPEDIA France.
DBPEDIA = SPARQLWrapper2("http://fr.dbpedia.org/sparql")

DBPEDIA_prefix = """
PREFIX dc:   <http://purl.org/dc/terms/>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX dbo:  <http://dbpedia.org/ontology/>

# Dans cette source, les noms de propriétés sont en français
PREFIX dbp:  <http://fr.dbpedia.org/property/>
"""
# print(DBPEDIA_prefix)


## Source 1 : La bibliothèque BNF


### BNF P1: Le nom des personnes

On suppose les relations :

- **CrééPar**(ouvrage, auteur)
- **Nom**(auteur, nom)

En SQL, la requête s'exprimerait ainsi:

> select distinct n.nom

> from CrééPar c, Nom n

> where c.auteur = n.auteur


In [7]:
requete = (
    BNF_prefix
    + """
SELECT distinct ?nom
WHERE {
  ?ouvrage dc:creator ?auteur .       # un ouvrage est créé par au moins un auteur
  ?auteur foaf:name ?nom              # un auteur a un nom
}
LIMIT 5
"""
)

BNF.setQuery(requete)
resultats = BNF.query().bindings
for r in tableauResultat(resultats):
    print(r)


['Fernand Gilles Lagrange']
['Musée des beaux-arts. Rouen']
['Château-Musée. Dieppe, Seine-Maritime']
['Évelyne Brisou-Pellen']
['Philippe Delestre']


### BNF T1: liste de thèmes

#### Les _themes_ des oeuvres dont l'auteur est référencé dans dbpedia france

En SQL, cette requête pourrait s'exprimer sur les tables suivantes

**créé_par** (livre, auteur)

**type_d_expression** (livre, type_expre) -- exple de tpye d'expression: textuelle

**écrit_en** (livre, langue)

**sujet** (livre, thème)

**auteur_équivalent** (auteur, auteur_source2)

et une condition SQL serait : WHERE auteur_source2 like 'http://fr.dbpedia.org%'


In [8]:
requete = (
    BNF_prefix
    + """
SELECT distinct ?theme
WHERE {
  
  # une oeuvre est associée a (au moins) un auteur
  ?livre dc:creator ?auteur .

  # un livre est un ouvrage dont la forme d'expression est textuelle (te)
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .
  
  # livre écrit en langue française
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a au moins un theme
  ?livre bnf:subject ?theme.

  # l'auteur existe également dans une autre source,
  # on dit qu'il est "aligné" avec un autre instance du même individu.
  ?auteur owl:sameAs ?auteurDbpedia .

  # la référence externe "pointe" vers dbpedia
  FILTER ( regex(?auteurDbpedia, "http://fr.dbpedia.org")  )
 
}
ORDER BY ?theme
LIMIT 20
"""
)

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)

# Exemple:
# Informatique
# Langues
# Philosophie


0 éléments
schéma:  []


""


### BNF A1 : une liste d'auteurs

A1 : Liste des auteurs de livres écrits en français et nés pendant une certaine période, par exemple au 20eme siècle. Afficher le nom de l'auteur, son année et lieu de naissance.


In [9]:
requete = (
    BNF_prefix
    + """
SELECT distinct ?nom  ?naissance ?lieu
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom 
?auteur foaf:name ?nom .

# un auteur a une année et un lieu de naissance
?auteur bio:birth ?naissance.
?auteur voc2:placeOfBirth ?lieu

# on sélectionne les années de naissance dans l'intervalle [1900, 2000]
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer 
     && xsd:integer (?naissance) < '2000'^^xsd:integer )
}
#ORDER BY ?nom
LIMIT 20
"""
)

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)


20 éléments
schéma:  ['nom', 'naissance', 'lieu']


,nom,naissance,lieu
0,Nardo Zalko,1941,Buenos Aires (Argentine)
1,Guy Perrier,1925,Chepniers (Charente-Maritime)
2,Marie-Madeleine Machet,1910,Bordeaux
3,Gilberte Émile-Mâle,1912,Paris
4,Philippe Goujard,1946,Rouen (Seine-Maritime)
5,André Hastoy,1920,Paris (France)
6,Jeanne Hives,1927,Villeneuve-sur-Lot (Lot-et-Garonne)
7,Michel Grelon,1945,Angoulême (Chrente)
8,Odette Elina,1910,Paris
9,André Frère,1908,"Anvers, Belgique"


### BNF A2 : les auteurs de la BNF référencés dans dbpedia

Les auteurs ayant un identifiant externe pointant vers dbpedia


In [10]:
requete = (
    BNF_prefix
    + """
SELECT distinct ?nom  ?naissance ?auteurDbpedia
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom, et une année de naissance
?auteur foaf:name ?nom ; bio:birth ?naissance .

# l'auteur existe également dans une source externe,
# on dit qu'il est "aligné" avec un autre instance du même individu.
?auteur owl:sameAs ?auteurDbpedia .

# naissance dans l'intervalle [1900, 2000] et la référence externe "pointe" vers dbpedia
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer 
     && xsd:integer (?naissance) < '2000'^^xsd:integer 
     && regex(?auteurDbpedia, "http://fr.dbpedia.org") 
     )
}
ORDER BY desc(?naissance)
LIMIT 10
"""
)

BNF.setQuery(requete)

resultats = BNF.query().bindings
afficherResultat(resultats)


10 éléments
schéma:  ['nom', 'naissance', 'auteurDbpedia']


,nom,naissance,auteurDbpedia
0,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
1,Isabelle Périer,1978,http://fr.dbpedia.org/resource/Isabelle_P%C3%A9rier
2,Nathanaël Dupré La Tour,1977,http://fr.dbpedia.org/resource/Nathana%C3%ABl_Dupr%C3%A9_La_Tour
3,Coralie Delaume,1976,http://fr.dbpedia.org/resource/Coralie_Delaume
4,Claire Guézengar,1972,http://fr.dbpedia.org/resource/Claire_Guezengar
5,Philippe Nassif,1971,http://fr.dbpedia.org/resource/Philippe_Nassif
6,Benoït Violier,1971,http://fr.dbpedia.org/resource/Beno%C3%AEt_Violier
7,Pierre Cherruau,1969,http://fr.dbpedia.org/resource/Pierre_Cherruau
8,Emmanuelle Marie,1965,http://fr.dbpedia.org/resource/Emmanuelle_Marie
9,Thierry Wanegffelen,1965,http://fr.dbpedia.org/resource/Thierry_Wanegffelen


### BNF A3 : Les auteurs d'une oeuvre philosophique

Voir T1 pour la liste des thèmes possibles autres que la philosophie.

On se limite aux oeuvres écrite en français et sont l'auteur est référencé dans dbpedia france.


In [11]:
requete = (
    BNF_prefix
    + """
SELECT distinct ?nom ?auteurDbpedia 
WHERE { 
  
  ?livre dc:creator ?auteur .

  # un livre est un ouvrage textuel
  #?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a pour theme "Philosophie"
  ?livre bnf:subject "Philosophie" .

  # le nom de l'auteur
  ?auteur foaf:name ?nom .

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER ( regex(?livreWiki, "http://wikidata.org") 
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?auteurDbpedia

LIMIT 20
"""
)

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats, 5)


20 éléments
schéma:  ['nom', 'auteurDbpedia']


,nom,auteurDbpedia
0,Alexis Carrel,http://fr.dbpedia.org/resource/Alexis_Carrel
1,Antoine de Rivarol,http://fr.dbpedia.org/resource/Antoine_de_Rivarol
2,Bernard-Henri Lévy,http://fr.dbpedia.org/resource/Bernard-Henri_L%C3%A9vy
3,Bernard de Fontenelle,http://fr.dbpedia.org/resource/Bernard_Le_Bouyer_de_Fontenelle
4,Claude Guillon,http://fr.dbpedia.org/resource/Claude_Guillon_%28%C3%A9crivain%29


### BNF L1 : Les livres avec leurs auteurs

Les livres écrit en français au 21eme siècle, référencés dans wikidata et dont l'auteur est référencé dans dbpedia


In [12]:
requete = (
    BNF_prefix
    + """
SELECT distinct ?annee ?nom ?titre ?auteurDbpedia
WHERE { 
  
  ?livre dc:creator ?auteur .

  # l'année de parution du livre
  ?livre dc:date ?annee .

  # le titre du livre
  ?livre dc:title ?titre .

  # un livre est un ouvrage textuel
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le nom de l'auteur
  ?auteur foaf:name ?nom.

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER (
       xsd:integer (?annee) > '2000'^^xsd:integer 
    && xsd:integer (?annee) < '2099'^^xsd:integer 
    && regex(?livreWiki, "http://wikidata.org") 
    &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
#ORDER BY desc(?annee)

LIMIT 20
"""
)

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)


0 éléments
schéma:  []


""


## Source 2 : DBPedia France


### D1 : Informations sur une personne dont le nom contient certains mots.

Afficher le nom, nationalité, profession, date et lieu de naissance


In [13]:
# l'expression régulière du nom à retrouver
MOTIF = "Victor.*Hug"

requete_parametree = (
    DBPEDIA_prefix
    + """
  SELECT ?p ?name ?nationalité ?profession year(xsd:dateTime(?naissance)) as ?naissance  ?lieu
  WHERE { 
    ?p a dbo:Person .
    ?p foaf:name ?name .
    ?p dbo:birthDate ?naissance
    optional {?p dbp:nationalité ?nationalité}
    optional {?p dbp:profession ?profession}
    optional {?p dbp:lieuDeNaissance ?lieu}

  FILTER (regex(?p, "{MOTIF}", "i") )
  }
  LIMIT 10
"""
)

requete = requete_parametree.replace("{MOTIF}", MOTIF)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)


10 éléments
schéma:  ['p', 'name', 'nationalité', 'profession', 'naissance', 'lieu']


,p,name,nationalité,profession,naissance,lieu
0,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
1,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
2,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
3,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
4,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
5,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
6,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
7,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
8,http://fr.dbpedia.org/resource/Victor_Hugo_Green,Victor Hugo Green,Américain,,1892,http://fr.dbpedia.org/resource/New_York
9,http://fr.dbpedia.org/resource/Victor_Hugo_Green,Victor Hugo Green,Américain,,1892,http://fr.dbpedia.org/resource/États-Unis


### D2 : les infos détaillées sur une certaine personne


In [14]:
# On fixe une certaine personne
PERSONNE = "http://fr.dbpedia.org/resource/Simone_de_Beauvoir"
# PERSONNE = 'http://fr.dbpedia.org/resource/Jean-Paul_Sartre'

# substituer une variable par sa valeur dans une chaine de caractères
# id = f"<{PERSONNE}>"
id = "<{PERSONNE}>".replace("{PERSONNE}", PERSONNE)
print("l'identifiant complet de la personne recherchée est :", id)

requete_parametree = (
    DBPEDIA_prefix
    + """
  SELECT distinct ?name ?nationalite ?interet ?idee
  WHERE { 
    ?pers foaf:name ?name 
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

#      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
#      optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> ) 
  }
  order by ?p
  LIMIT 20
"""
)

# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

# print("la requête est :", requete)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)


l'identifiant complet de la personne recherchée est : <http://fr.dbpedia.org/resource/Simone_de_Beauvoir>
6 éléments
schéma:  ['name', 'interet', 'idee']


,name,interet,idee
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Philosophie,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Politique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Éthique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
3,,http://fr.dbpedia.org/resource/Philosophie,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
4,,http://fr.dbpedia.org/resource/Politique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
5,,http://fr.dbpedia.org/resource/Éthique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"


### D3 : la liste des oeuvres principales d'une personne donnée


In [15]:
# On fixe une certaine personne
PERSONNE = "http://fr.dbpedia.org/resource/Simone_de_Beauvoir"

requete_parametree = (
    DBPEDIA_prefix
    + """
    SELECT distinct ?name ?oeuvre
    WHERE { 
      ?pers foaf:name ?name 
      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
#     optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> ) 
}
order by ?p
LIMIT 20
"""
)
# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)


10 éléments
schéma:  ['name', 'oeuvre']


,name,oeuvre
0,Simone de Beauvoir,L'Invitée
1,Simone de Beauvoir,La Force de l'âge
2,Simone de Beauvoir,Le Deuxième Sexe
3,Simone de Beauvoir,Les Mandarins
4,Simone de Beauvoir,Mémoires d'une jeune fille rangée
5,,L'Invitée
6,,La Force de l'âge
7,,Le Deuxième Sexe
8,,Les Mandarins
9,,Mémoires d'une jeune fille rangée


### D4 : les influenceurs d'une personne donnée


In [16]:
# On fixe une certaine personne
PERSONNE = "http://fr.dbpedia.org/resource/Simone_de_Beauvoir"
# PERSONNE = 'http://fr.dbpedia.org/resource/Judith_Butler'

requete_parametree = (
    DBPEDIA_prefix
    + """
   SELECT distinct ?name ?influenceur
    WHERE { 
      ?pers foaf:name ?name 
      optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> ) 
}
order by ?p
LIMIT 20
"""
)

# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
print(len(res), "resultats")
afficherResultat(res)


20 resultats
20 éléments
schéma:  ['name', 'influenceur']


,name,influenceur
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Alfred_Adler
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Edmund_Husserl
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Emmanuel_Kant
3,Simone de Beauvoir,http://fr.dbpedia.org/resource/François_Poullain_de_La_Barre
4,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Engels
5,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Nietzsche
6,Simone de Beauvoir,http://fr.dbpedia.org/resource/Georg_Wilhelm_Friedrich_Hegel
7,Simone de Beauvoir,http://fr.dbpedia.org/resource/Jean-Paul_Sartre
8,Simone de Beauvoir,http://fr.dbpedia.org/resource/Karl_Marx
9,Simone de Beauvoir,http://fr.dbpedia.org/resource/René_Descartes


### D5 : Les auteurs nés en 1950

Utiliser la fonction year pour extraire l'année de naissance à partir d'une date de naissance.


In [17]:
# On fixe une certaine année
ANNEE = 1951

requete_parametree = (
    DBPEDIA_prefix
    + """
    SELECT ?p ?name year(xsd:dateTime(?naissance)) as ?naissance
    WHERE { 
      ?p a dbo:Person .
      ?p foaf:name ?name .
      ?p dbo:birthDate ?naissance
      FILTER( regex(?naissance, "{ANNEE}"))
    }
order by ?p
LIMIT 10
"""
)

requete = requete_parametree.replace("{ANNEE}", str(ANNEE))

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)


10 éléments
schéma:  ['p', 'name', 'naissance']


,p,name,naissance
0,http://fr.dbpedia.org/resource/Abdallah_Kaâbi,,1951
1,http://fr.dbpedia.org/resource/Abdallah_Kaâbi,Abdallah Kaâbi,1951
2,http://fr.dbpedia.org/resource/Abdallah_Kaâbi,,1951
3,http://fr.dbpedia.org/resource/Abdallah_Kaâbi,Abdallah Kaâbi,1951
4,http://fr.dbpedia.org/resource/Abdelatif_Baba_Ahmed,,1951
5,http://fr.dbpedia.org/resource/Abdelatif_Baba_Ahmed,Abdelatif Baba Ahmed,1951
6,http://fr.dbpedia.org/resource/Abdelatif_Baba_Ahmed,,1951
7,http://fr.dbpedia.org/resource/Abdelatif_Baba_Ahmed,Abdelatif Baba Ahmed,1951
8,http://fr.dbpedia.org/resource/Abdeslam_Seddiki,,1951
9,http://fr.dbpedia.org/resource/Abdeslam_Seddiki,Abdeslam Seddiki,1951


### D6 : La liste de tous les intérets principaux


In [18]:
requete = (
    DBPEDIA_prefix
    + """
    SELECT distinct ?interet
    WHERE { 
      ?personne dbo:mainInterest ?o .
      ?o rdfs:label ?interet .

      FILTER (langMatches( lang(?interet), "FR" ) )
}
order by ?interet
LIMIT 50
"""
)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)


50 éléments
schéma:  ['interet']


,interet
0,Activisme politique
1,Afrique
2,Agnosticisme
3,Alchimie
4,Amour
5,Amour platonique
6,Angélologie
7,Anthropologie
8,Anthropologie linguistique
9,Anthropologie philosophique


### D7 la liste des philosophes dans Dbpedia


In [19]:
import time

requete = (
    DBPEDIA_prefix
    + """
    
SELECT distinct  ?name ?p
WHERE { 
  ?p rdf:type dbo:Philosopher .
  ?p foaf:name ?name
}
order by ?name
limit 10
"""
)

start_time = time.time()

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings

duree = int((time.time() - start_time) * 1000)
print(f"durée {duree} ms")

afficherResultat(res)


durée 126 ms
10 éléments
schéma:  ['name', 'p']


,name,p
0,,http://fr.dbpedia.org/resource/Œuvre_de_Friedrich_Nietzsche
1,,http://fr.dbpedia.org/resource/Abu_'Ali_Al-Jubbâ'i
2,,http://fr.dbpedia.org/resource/Abu_al-Hudhayl
3,,http://fr.dbpedia.org/resource/Abū_Hāshīm_al-Jubbā'ī
4,,http://fr.dbpedia.org/resource/Adam_Smith
5,,http://fr.dbpedia.org/resource/Aimé_Forest
6,,http://fr.dbpedia.org/resource/Al-Fârâbî
7,,http://fr.dbpedia.org/resource/Al-Ghazali
8,,http://fr.dbpedia.org/resource/Alain_(philosophe)
9,,http://fr.dbpedia.org/resource/Alfred_Tarski


# Exercice 1 : Requêtes réparties entre les 2 sources BNF et DBPedia

A compléter


### J1: Jointure A3 avec D2 par boucles imbriquées

Proposer une solution pour calculer la jointure entre A3 sur le site BNF et D2 sur le site DBPedia.
Pour chaque livre de afficher son titre et son année, le nom de l'auteur, sa nationalité et la liste de ses idées remarquables.


In [89]:
requete = (
    BNF_prefix
    + """
SELECT distinct ?livre ?nom ?naissance ?auteurDbpedia
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom, et une année de naissance
?auteur foaf:name ?nom ; bio:birth ?naissance .

# l'auteur existe également dans une source externe,
# on dit qu'il est "aligné" avec un autre instance du même individu.
?auteur owl:sameAs ?auteurDbpedia .

# naissance dans l'intervalle [1900, 2000] et la référence externe "pointe" vers dbpedia
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer 
     && xsd:integer (?naissance) < '2000'^^xsd:integer 
     && regex(?auteurDbpedia, "http://fr.dbpedia.org") 
     )
}
ORDER BY desc(?naissance)
LIMIT 1000
"""
)

BNF.setQuery(requete)

res_A3 = BNF.query().bindings
df_A3 = pd.DataFrame(res_A3).applymap(lambda x: x.value)

In [92]:
df_A3

,livre,nom,naissance,auteurDbpedia
0,http://data.bnf.fr/temp-work/cfaedd0417cc942f2cba0522e60de1df/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
1,http://data.bnf.fr/temp-work/e7e18631c382c2ac1cb4865cd348c753/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
2,http://data.bnf.fr/temp-work/d6f5ce4eebd69d292068ff9f4f674512/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
3,http://data.bnf.fr/temp-work/00ed4610432378cac7b1d33464e51352/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
4,http://data.bnf.fr/temp-work/dda86386c8a378808e05b81b53424a1a/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
...,...,...,...,...
6674,http://data.bnf.fr/temp-work/7b3ac660e537b35e50c609685393b2b6/#about,Pierre Morhange,1901,http://fr.dbpedia.org/resource/Pierre_Morhange
6675,http://data.bnf.fr/temp-work/667cfc6838321294aad921e4ff1ee71f/#about,Pierre Morhange,1901,http://fr.dbpedia.org/resource/Pierre_Morhange
6676,http://data.bnf.fr/temp-work/06c80a0dd179e6c63ccab52c4e8d5294/#about,Pierre Biquard,1901,http://fr.dbpedia.org/resource/Pierre_Biquard
6677,http://data.bnf.fr/temp-work/4c7005074762ded79b1611eaee8e0f18/#about,Pierre Biquard,1901,http://fr.dbpedia.org/resource/Pierre_Biquard


In [93]:
personnes = df_A3["auteurDbpedia"].unique()
len(personnes)

['http://fr.dbpedia.org/resource/Axl_Cendres'
 'http://fr.dbpedia.org/resource/Isabelle_P%C3%A9rier'
 'http://fr.dbpedia.org/resource/Nathana%C3%ABl_Dupr%C3%A9_La_Tour' ...
 'http://fr.dbpedia.org/resource/Amadou_Hamp%C3%A2t%C3%A9_B%C3%A2'
 'http://fr.dbpedia.org/resource/Pierre_Morhange'
 'http://fr.dbpedia.org/resource/Pierre_Biquard']


In [94]:
requete_parametree = (
    DBPEDIA_prefix
    + """
  SELECT distinct ?name ?nationalite ?interet ?idee
  WHERE { 
    ?pers foaf:name ?name 
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

#      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
#      optional {?pers  dbo:influencedBy ?influenceur}

   auteur FILTER ( ?pers = <{PERSONNE}> ) 
  }
  order by ?p
  LIMIT 20
"""
)

res = []
for personne in personnes:
    DBPEDIA.setQuery(requete_parametree.replace("{PERSONNE}", personne))
    D2 = DBPEDIA.query().bindings
    res += D2

afficherResultat(res)


257 éléments
schéma:  ['name', 'nationalite']


,name,nationalite
0,,
1,Philippe Nassif,
2,,
3,Thierry Aprile,
4,,
...,...,...
95,John Littleton,
96,,
97,Michel van Schendel,
98,,


### J2 : Pour une année N afficher la liste des personnes qui sont nées l'année N et qui existent à la fois dans la BNF et dans DBPedia.

Proposer une solution par fusion de deux résultats triés


In [127]:
# A compléter
annee = 1908

req1 = (
    BNF_prefix
    + """
SELECT distinct ?nom ?auteurDbpedia ?naissance
WHERE {
# un auteur a un nom, et une année de naissance
?auteur foaf:name ?nom ; bio:birth ?naissance .

# l'auteur existe également dans une source externe,
# on dit qu'il est "aligné" avec un autre instance du même individu.
?auteur owl:sameAs ?auteurDbpedia .

# référence externe "pointe" vers dbpedia
FILTER (regex(?auteurDbpedia, "http://fr.dbpedia.org")
        && xsd:integer(?naissance) = '{ANNEE}'^^xsd:integer )
}
ORDER BY desc(?naissance)
LIMIT 1000
"""
)

BNF.setQuery(req1.replace("{ANNEE}", str(annee)))
res1 = BNF.query().bindings
df1 = pd.DataFrame(res1).applymap(lambda x: x.value)

afficherResultat(res1)


127 éléments
schéma:  ['nom', 'auteurDbpedia', 'naissance']


,nom,auteurDbpedia,naissance
0,André Baruc,http://fr.dbpedia.org/resource/Andr%C3%A9_Baruc,1908
1,Paul van Stalle,http://fr.dbpedia.org/resource/Paul_Van_Stalle,1908
2,Harold Rome,http://fr.dbpedia.org/resource/Harold_Rome,1908
3,Jane Chacun,http://fr.dbpedia.org/resource/Jane_Chacun,1908
4,Georg Hurdalek,http://fr.dbpedia.org/resource/George_Hurdalek,1908
...,...,...,...
95,Nils Johan Rud,http://fr.dbpedia.org/resource/Nils_Johan_Rud,1908
96,Germaine Lacaze,http://fr.dbpedia.org/resource/Germaine_Lacaze,1908
97,Peter Charanis,http://fr.dbpedia.org/resource/Peter_Charanis,1908
98,Jeannie Robertson,http://fr.dbpedia.org/resource/Jeannie_Robertson,1908


In [128]:

req2 = (
    DBPEDIA_prefix
    + """
  SELECT distinct ?p year(xsd:dateTime(?naissance)) as ?naissance
  WHERE { 
    ?p a dbo:Person .
    ?p dbo:birthDate ?naissance
    FILTER ( regex (?naissance, '{ANNEE}') )
  }
  ORDER BY desc(?naissance)
  LIMIT 10000
"""
)

DBPEDIA.setQuery(req2.replace("{ANNEE}", str(annee)))
res2 = DBPEDIA.query().bindings
df2 = pd.DataFrame(res2).applymap(lambda x: x.value)

afficherResultat(res2)


1291 éléments
schéma:  ['p', 'naissance']


,p,naissance
0,http://fr.dbpedia.org/resource/Štefan_Čambal,1908
1,http://fr.dbpedia.org/resource/Ștefan_Barbu,1908
2,http://fr.dbpedia.org/resource/A.I._Bezzerides,1908
3,http://fr.dbpedia.org/resource/Abdelhalim_Hemche,1908
4,http://fr.dbpedia.org/resource/Abdelkader_Benazzedine,1908
...,...,...
95,http://fr.dbpedia.org/resource/Antoine_Mazier,1908
96,http://fr.dbpedia.org/resource/Antonio_Canepa,1908
97,http://fr.dbpedia.org/resource/Antonio_Cánovas_del_Castillo_de_Rey,1908
98,http://fr.dbpedia.org/resource/Antonín_Bartoň,1908


In [129]:
df1.merge(df2, left_on="auteurDbpedia", right_on="p")

,nom,auteurDbpedia,naissance_x,p,naissance_y
0,Harold Rome,http://fr.dbpedia.org/resource/Harold_Rome,1908,http://fr.dbpedia.org/resource/Harold_Rome,1908
1,Georg Hurdalek,http://fr.dbpedia.org/resource/George_Hurdalek,1908,http://fr.dbpedia.org/resource/George_Hurdalek,1908
2,Georgij Georgievič Demidov,http://fr.dbpedia.org/resource/Gueorgui_Demidov,1908,http://fr.dbpedia.org/resource/Gueorgui_Demidov,1908
3,Richard Wormser,http://fr.dbpedia.org/resource/Richard_Wormser,1908,http://fr.dbpedia.org/resource/Richard_Wormser,1908
4,Yvette Cauchois,http://fr.dbpedia.org/resource/Yvette_Cauchois,1908,http://fr.dbpedia.org/resource/Yvette_Cauchois,1908
5,Vicente Asencio,http://fr.dbpedia.org/resource/Vicente_Asencio,1908,http://fr.dbpedia.org/resource/Vicente_Asencio,1908
6,Robert Cummings,http://fr.dbpedia.org/resource/Robert_Cummings,1908,http://fr.dbpedia.org/resource/Robert_Cummings,1908
7,Jean Jeanneteau,http://fr.dbpedia.org/resource/Jean_Jeanneteau,1908,http://fr.dbpedia.org/resource/Jean_Jeanneteau,1908
8,Isis Kischka,http://fr.dbpedia.org/resource/Isis_Kischka,1908,http://fr.dbpedia.org/resource/Isis_Kischka,1908
9,Ralph Smart,http://fr.dbpedia.org/resource/Ralph_Smart,1908,http://fr.dbpedia.org/resource/Ralph_Smart,1908


### J3 : Quels sont les auteurs d'une oeuvre dont le thème est philosophie (dans la BNF) et qui sont référencés comme étant des philosophes dans DBpedia ?


In [22]:
# A compléter


# Exercice 2 : Accès aux données par des requêtes de sélections

L'objectif de cet exercice est d'appréhender la situation où l'accès aux données est contraint (voir la notion de [linked data fragment](https://linkeddatafragments.org/concept/)). Le point d'accès n'autorise que des requêtes simples (donc peut coûteuses à traiter) qui ne contiennent que des **sélections**. La clause where d'une requête ne peut contenir qu'**un seul** motif ayant 1 ou 2 variables. Les motifs autorisés sont donc parmi :

- ?a _propriété_ ?b _(cette requête a deux variables)_
- _A_ _propriété_ ?b _(cette requête a une seule variable)_
- ?a _propriété_ _B_

où ?a et ?b sont des variables et _A_, _B_ et _propriété_ sont des valeurs fixées avant de poser la requête.

Ainsi, toutes les jointures doivent être exprimées par le client qui accède aux données : vous devez "programmer" toutes les jointures en python.
Pour les requêtes D1 et D6 proposer une solution respectant ce mode d'accès.


In [23]:
# à compléter


# Exercice 3 : Accès par morceau au résultat d'une requête

1. Un utilisateur veut accéder aux données de manière progressive. Proposer une solution pour obtenir le resultat d'une requete par morceau de N tuples, en utilisant les mots-clés OFFSET et LIMIT. Appliquer cela pour lire le résultat de la requête qui affiche le nom des personnes existant dans dbpedia.
   Etudier l'influence de l'offset et de la taille du résultat sur le temps de réponse de la requête.


In [24]:
# Exemple d'utilisation de OFFSET et LIMIT

# Compléter cet exemple


import time

requete = (
    DBPEDIA_prefix
    + """

SELECT distinct ?name ?p 
WHERE { 
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
}
offset 33000 limit 10
"""
)

start_time = time.time()
# exécution de la requête
DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
duree = int((time.time() - start_time) * 1000)
print(f"la requête dure {duree} ms")

afficherResultat(res)


la requête dure 309 ms
10 éléments
schéma:  ['name', 'p']


,name,p
0,,http://fr.dbpedia.org/resource/Antonin_Galipeault
1,Antonin Galipeault,http://fr.dbpedia.org/resource/Antonin_Galipeault
2,,http://fr.dbpedia.org/resource/Antonin_Gourju
3,Antonin Gourju,http://fr.dbpedia.org/resource/Antonin_Gourju
4,,http://fr.dbpedia.org/resource/Antonin_Gros
5,Antonin Gros,http://fr.dbpedia.org/resource/Antonin_Gros
6,,http://fr.dbpedia.org/resource/Antonin_Guigonnat
7,Antonin Guigonnat,http://fr.dbpedia.org/resource/Antonin_Guigonnat
8,Antonin Lecouteux,http://fr.dbpedia.org/resource/Antonin_Lecouteux
9,,http://fr.dbpedia.org/resource/Antonin_Louchard


2. Définir une fonction traiteRequete(requete) qui retourne un generateur sur une requete de telle sorte que la requete soit lue par morceaux de N tuples et que seuls les morceaux permettant de produire les M lignes demandées par l'utlisateur soit lus. On ne connait pas à l'avance le nombre M de lignes que va lire l'utilisateur. On a M > N.
   Indication voir le mot clé python **yield** pour retourner un générateur.


In [25]:
requete = (
    DBPEDIA_prefix
    + """
SELECT distinct ?name ?p 
WHERE { 
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
}
offset 0 limit 10
"""
)


# définition d'un générateur
def traiteRequete(requete):
    DBPEDIA.setQuery(requete)
    res = DBPEDIA.query().bindings
    print(f"leture d'un morceau de {len(res)} lignes")
    for r in res:
        yield r


# Nombre de tuples que l'utilisateur souhaite lire
a_lire = 20

lu = 0
for a in traiteRequete(requete):
    print(a)
    lu += 1
    if lu >= a_lire:
        break

print()
if lu < a_lire:
    print(f"Attention !!!! l'utilisateur n'a obtenu que {lu} lignes sur les {a_lire}")
else:
    print(f"OK, l'utilisateur a obtenu {lu} lignes sur les {a_lire}")


leture d'un morceau de 10 lignes
{'name': Value(literal:''), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Bujko')}
{'name': Value(literal:'Łukasz Bujko'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Bujko')}
{'name': Value(literal:''), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Gibała')}
{'name': Value(literal:'Łukasz Gibała'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Gibała')}
{'name': Value(literal:''), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Gierak')}
{'name': Value(literal:'Gierak'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Gierak')}
{'name': Value(literal:''), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Grzeszczuk')}
{'name': Value(literal:'Łukasz Grzeszczuk'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Grzeszczuk')}
{'name': Value(literal:''), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Kaczmarek')}
{'name': Value(literal:'Łukasz Kaczmarek'), 'p': Value(uri:'http://fr.dbpedia.org/

3. Proposer une solution pour accéder de manière progressive au résultat d'une requête répartie entre 2 sites, par exemple J1.


In [26]:
# à compléter


# Exerice 4 : Cache de requêtes

1. Proposer des améliorations des requêtes précédentes afin d'éviter de poser plusieurs fois une même requête. On suppose que le résultat de la requête ne change pas entre deux invocations de la même requête.

2. Cache avec une capacité limitée : considérer qu'on peut conserver en cache le résultat de C requêtes (C=10) au maximum


In [27]:
# A compléter


# FAQ: Questions diverses


## Mise au point des requêtes

Pour débugguer et **mettre au point vos requêtes** Sparql, il est utile de les tester directement sur l'interface du point d'accès dédié de la source de données. L'avantage est que cela vous retourne un message indiquant très précisément la nature et l'endroit où se trouve l'erreur dans la requête.

- [BNF](http://data.bnf.fr/sparql) voir des [exemples](https://api.bnf.fr/sparql-endpoint-de-databnffr)
- [DBPedia](http://fr.dbpedia.org/sparql) et la liste de tous les [prefixes](http://fr.dbpedia.org/sparql?nsdecl)


## Autres sources de données

Autres sources à votre disposition pour proposer des requêtes multi-sources:

1.  [Mondial](http://www.semwebtech.org/mondial/10/sparql) : données géopolitiques. Attention le résulat des requêtes est en XML pas en JSON.
2.  [Persée](data.persee.fr) : données bibliographiques alignées avec plusieurs autres sources dont dbpedia. Voir des [exemples](http://data.persee.fr/ressources/requetes-exemples/)
3.  [BioPortal](http://sparql.bioontology.org)
4.  [IdRef](https://data.idref.fr/sparql) serveur d’autorité IdRef et les références bibliographiques en provenance du Sudoc.
5.  [Insee](https://rdf.insee.fr/sparql)
6.  [Geonames web services](https://www.geonames.org/export/ws-overview.html)


## SPARKLIS : aide à l'écriture de requêtes Sparql

Pour exprimer vos requêtes en _langage naturel_ et les traduire automatiquement en SPARQL, voir l'appli [Sparklis](http://www.irisa.fr/LIS/ferre/sparklis/)

- onglet **Sparklis** pour exprimer vos requêtes en langage naturel
- onglet **YASGUI** pour voir la traduction en Sparql


## Nombre d'éléments dans le resultat d'une requête

Pour compter le nombre d'éléments d'une requête, utiliser la notation avec requête imbriquée dans le where :

SELECT (count(1) as ?nb)

WHERE { _sous-requete_ }
